In [3]:
data_dir = "./covid19-global-forecasting-week-1/"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import tensorflow as tf
import math

# Read dataset

In [4]:
train_df_raw = pd.read_csv(data_dir+"train.csv")
test_df_raw = pd.read_csv(data_dir+"test.csv")
all_df = train_df_raw
test_df = test_df_raw

In [5]:
# paramerers
maxlen = 30
hidden_number = 32
input_number = 5
output_number = 2
batch_size = 32
epochs = 50
lr = 0.001

# Preprocessing

I Normalize the numeric values and make date column to the numeric values.   
The final day of the training dataset is used as the validation data.

In [6]:
def map_datetime(date):
    return (date - datetime.datetime.strptime('2020-01-22', "%Y-%m-%d")).days

def to_datetime(date):
    return datetime.datetime.strptime(date, "%Y-%m-%d")

In [7]:
test_df = test_df.fillna({"Province/State": "NAN"})
all_df = train_df.fillna({"Province/State": "NAN"})

all_df["ConfirmedCases"] = (all_df["ConfirmedCases"] + 1).map(math.log10)
cases_max = all_df["ConfirmedCases"].max()
fatal_max = all_df["Fatalities"].max()

all_df["Lat"] = all_df["Lat"]/180.
all_df["Long"] = all_df["Long"]/180.
all_df["ConfirmedCases"] = all_df["ConfirmedCases"] / cases_max
all_df["Fatalities"] = all_df["Fatalities"] / fatal_max

all_df["Date"] = all_df["Date"].map(to_datetime)
all_df["Date_num"] = all_df["Date"].map(map_datetime)
date_max = all_df["Date_num"].max()
all_df["Date_num"] = all_df["Date_num"] / date_max

date_unit = all_df.iloc[1]["Date_num"] - all_df.iloc[0]["Date_num"]

val_df = all_df[all_df["Date"] > (all_df["Date"].max() - datetime.timedelta(days=(maxlen+1)))]
train_df = all_df.drop(all_df[all_df["Date_num"] == all_df["Date_num"].max()].index)

test dataframe is also preprocessed

In [8]:
test_df["Lat"] = test_df["Lat"]/180.
test_df["Long"] = test_df["Long"]/180.
test_df["Date"] = test_df["Date"].map(to_datetime)
test_df["Date_num"] = test_df["Date"].map(map_datetime)
test_df["Date_num"] = test_df["Date_num"] / date_max

# Preprocessing for LSTM

Reshape the dataset to sequences.  

Input data components are Latitude, Longnitude, Date, ConfirmedCases, Fatalities.  
Output data components are ConfirmedCases, Fatalities.  


In [9]:
def make_sequences(train_df):
    inputs = []
    targets = []
    for i in range(len(train_df) - maxlen - 1):
        if train_df.iloc[i]["Lat"] == train_df.iloc[i+maxlen]["Lat"] and \
           train_df.iloc[i]["Long"] == train_df.iloc[i+maxlen]["Long"]:
            inputs.append(np.array(train_df.iloc[i:i+maxlen][["Date_num", "Lat", "Long", "ConfirmedCases", "Fatalities"]]).tolist())
            targets.append(np.array(train_df.iloc[i+maxlen][["ConfirmedCases", "Fatalities"]]).tolist())
    return inputs, targets

In [10]:
train_inputs, train_targets = make_sequences(train_df)
val_inputs, val_targets = make_sequences(val_df)

# Make LSTM model
In this section, I made two layered simple LSTM model.

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(hidden_number, batch_input_shape=[None, maxlen, input_number], return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_number))
model.add(tf.keras.layers.Dense(output_number, activation="sigmoid"))

optimizer = tf.keras.optimizers.Adam(lr=lr)
model.compile(loss="mean_squared_error", optimizer=optimizer)

In [12]:
model.load_weights("./tmp/param.hdf5")

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', patience=5)
history = model.fit(train_inputs, train_targets,
                    batch_size=batch_size,
                      epochs=epochs,
                      validation_data=(val_inputs, val_targets),
                      callbacks = [early_stopping]
                      )

# Prediction phase

In [13]:

all_df

Predict 43 days recursively by many-to-one LSTM model.  
The prediction step is one step. So, the prediction is recursively continued to predict 43 days cases.

In [70]:
results = []
for idx in test_df.groupby(["Province/State", "Country/Region"]).count().index:
    test_df_on_idx = test_df[(test_df["Province/State"] == idx[0]) &
                             (test_df["Country/Region"] == idx[1])]
    train_df_on_idx = train_df[(all_df["Country/Region"] == idx[1]) &
                               (all_df["Province/State"] == idx[0])]
    inputs = np.array(train_df_on_idx[["Date_num", "Lat", "Long", "ConfirmedCases", "Fatalities"]])[-maxlen:]
    inputs = inputs.reshape(maxlen, input_number)
    for day in range(43):
        if int(1000000000*(datetime.timedelta(days=day) + test_df_on_idx["Date"].min()).timestamp()) in train_df_on_idx["Date"].values.tolist():
            result = np.array(train_df_on_idx[train_df_on_idx["Date"] == (datetime.timedelta(days=day) + test_df_on_idx["Date"].min())][["Date", "Lat", "Long", "ConfirmedCases", "Fatalities"]])[0, 3:]
        else:
            result = model.predict(np.array(inputs).reshape(1, maxlen, input_number)).reshape(-1)
            inputs = np.concatenate((inputs[1:], np.append(inputs[-1, :3], result).reshape(1, input_number)), axis=0)
        results.append([10**(result[0]*cases_max), result[1]*fatal_max])

find the day in teacher
[0.0 0.0]
find the day in teacher
[0.16106669895297376 0.0]
find the day in teacher
[0.17492377160560507 0.0]
find the day in teacher
[0.23057108561891348 0.0]
find the day in teacher
[0.30574395958841427 0.0]
find the day in teacher
[0.3316045745613166 0.0]
find the day in teacher
[0.3461014580022392 0.0]
find the day in teacher
[0.3927829342781318 0.0]
[[ 0.48275862  0.17954556 -0.48279056  0.          0.        ]
 [ 0.5         0.17954556 -0.48279056  0.          0.        ]
 [ 0.51724138  0.17954556 -0.48279056  0.          0.        ]
 [ 0.53448276  0.17954556 -0.48279056  0.          0.        ]
 [ 0.55172414  0.17954556 -0.48279056  0.          0.        ]
 [ 0.56896552  0.17954556 -0.48279056  0.          0.        ]
 [ 0.5862069   0.17954556 -0.48279056  0.          0.        ]
 [ 0.60344828  0.17954556 -0.48279056  0.          0.        ]
 [ 0.62068966  0.17954556 -0.48279056  0.          0.        ]
 [ 0.63793103  0.17954556 -0.48279056  0.          0

[0.00305647 0.00226174]
[[ 0.5862069   0.17954556 -0.48279056  0.          0.        ]
 [ 0.60344828  0.17954556 -0.48279056  0.          0.        ]
 [ 0.62068966  0.17954556 -0.48279056  0.          0.        ]
 [ 0.63793103  0.17954556 -0.48279056  0.          0.        ]
 [ 0.65517241  0.17954556 -0.48279056  0.          0.        ]
 [ 0.67241379  0.17954556 -0.48279056  0.          0.        ]
 [ 0.68965517  0.17954556 -0.48279056  0.          0.        ]
 [ 0.70689655  0.17954556 -0.48279056  0.          0.        ]
 [ 0.72413793  0.17954556 -0.48279056  0.          0.        ]
 [ 0.74137931  0.17954556 -0.48279056  0.          0.        ]
 [ 0.75862069  0.17954556 -0.48279056  0.          0.        ]
 [ 0.77586207  0.17954556 -0.48279056  0.          0.        ]
 [ 0.79310345  0.17954556 -0.48279056  0.          0.        ]
 [ 0.81034483  0.17954556 -0.48279056  0.          0.        ]
 [ 0.82758621  0.17954556 -0.48279056  0.          0.        ]
 [ 0.84482759  0.17954556 -0.48

[0.00305308 0.00225825]
[[ 0.67241379  0.17954556 -0.48279056  0.          0.        ]
 [ 0.68965517  0.17954556 -0.48279056  0.          0.        ]
 [ 0.70689655  0.17954556 -0.48279056  0.          0.        ]
 [ 0.72413793  0.17954556 -0.48279056  0.          0.        ]
 [ 0.74137931  0.17954556 -0.48279056  0.          0.        ]
 [ 0.75862069  0.17954556 -0.48279056  0.          0.        ]
 [ 0.77586207  0.17954556 -0.48279056  0.          0.        ]
 [ 0.79310345  0.17954556 -0.48279056  0.          0.        ]
 [ 0.81034483  0.17954556 -0.48279056  0.          0.        ]
 [ 0.82758621  0.17954556 -0.48279056  0.          0.        ]
 [ 0.84482759  0.17954556 -0.48279056  0.          0.        ]
 [ 0.86206897  0.17954556 -0.48279056  0.          0.        ]
 [ 0.87931034  0.17954556 -0.48279056  0.1610667   0.        ]
 [ 0.89655172  0.17954556 -0.48279056  0.17492377  0.        ]
 [ 0.9137931   0.17954556 -0.48279056  0.23057109  0.        ]
 [ 0.93103448  0.17954556 -0.48

[0.00305111 0.00225661]
[[ 0.79310345  0.17954556 -0.48279056  0.          0.        ]
 [ 0.81034483  0.17954556 -0.48279056  0.          0.        ]
 [ 0.82758621  0.17954556 -0.48279056  0.          0.        ]
 [ 0.84482759  0.17954556 -0.48279056  0.          0.        ]
 [ 0.86206897  0.17954556 -0.48279056  0.          0.        ]
 [ 0.87931034  0.17954556 -0.48279056  0.1610667   0.        ]
 [ 0.89655172  0.17954556 -0.48279056  0.17492377  0.        ]
 [ 0.9137931   0.17954556 -0.48279056  0.23057109  0.        ]
 [ 0.93103448  0.17954556 -0.48279056  0.30574396  0.        ]
 [ 0.94827586  0.17954556 -0.48279056  0.33160457  0.        ]
 [ 0.96551724  0.17954556 -0.48279056  0.34610146  0.        ]
 [ 0.98275862  0.17954556 -0.48279056  0.39278293  0.        ]
 [ 0.98275862  0.17954556 -0.48279056  0.00307284  0.00227016]
 [ 0.98275862  0.17954556 -0.48279056  0.00306128  0.00226741]
 [ 0.98275862  0.17954556 -0.48279056  0.00305924  0.00226543]
 [ 0.98275862  0.17954556 -0.48

[0.00305145 0.00225623]
[[ 0.87931034  0.17954556 -0.48279056  0.1610667   0.        ]
 [ 0.89655172  0.17954556 -0.48279056  0.17492377  0.        ]
 [ 0.9137931   0.17954556 -0.48279056  0.23057109  0.        ]
 [ 0.93103448  0.17954556 -0.48279056  0.30574396  0.        ]
 [ 0.94827586  0.17954556 -0.48279056  0.33160457  0.        ]
 [ 0.96551724  0.17954556 -0.48279056  0.34610146  0.        ]
 [ 0.98275862  0.17954556 -0.48279056  0.39278293  0.        ]
 [ 0.98275862  0.17954556 -0.48279056  0.00307284  0.00227016]
 [ 0.98275862  0.17954556 -0.48279056  0.00306128  0.00226741]
 [ 0.98275862  0.17954556 -0.48279056  0.00305924  0.00226543]
 [ 0.98275862  0.17954556 -0.48279056  0.0030582   0.00226401]
 [ 0.98275862  0.17954556 -0.48279056  0.00305733  0.00226281]
 [ 0.98275862  0.17954556 -0.48279056  0.00305647  0.00226174]
 [ 0.98275862  0.17954556 -0.48279056  0.00305566  0.00226082]
 [ 0.98275862  0.17954556 -0.48279056  0.00305491  0.00226002]
 [ 0.98275862  0.17954556 -0.48

...

[0.00305962 0.00226294]
[[ 0.63793103  0.18738778 -0.61906222  0.          0.        ]
 [ 0.65517241  0.18738778 -0.61906222  0.          0.        ]
 [ 0.67241379  0.18738778 -0.61906222  0.          0.        ]
 [ 0.68965517  0.18738778 -0.61906222  0.          0.        ]
 [ 0.70689655  0.18738778 -0.61906222  0.          0.        ]
 [ 0.72413793  0.18738778 -0.61906222  0.          0.        ]
 [ 0.74137931  0.18738778 -0.61906222  0.          0.        ]
 [ 0.75862069  0.18738778 -0.61906222  0.          0.        ]
 [ 0.77586207  0.18738778 -0.61906222  0.          0.        ]
 [ 0.79310345  0.18738778 -0.61906222  0.          0.        ]
 [ 0.81034483  0.18738778 -0.61906222  0.          0.        ]
 [ 0.82758621  0.18738778 -0.61906222  0.17492377  0.        ]
 [ 0.84482759  0.18738778 -0.61906222  0.20698637  0.        ]
 [ 0.86206897  0.18738778 -0.61906222  0.20698637  0.        ]
 [ 0.87931034  0.18738778 -0.61906222  0.20698637  0.        ]
 [ 0.89655172  0.18738778 -0.61

[0.00305646 0.0022606 ]
[[ 0.75862069  0.18738778 -0.61906222  0.          0.        ]
 [ 0.77586207  0.18738778 -0.61906222  0.          0.        ]
 [ 0.79310345  0.18738778 -0.61906222  0.          0.        ]
 [ 0.81034483  0.18738778 -0.61906222  0.          0.        ]
 [ 0.82758621  0.18738778 -0.61906222  0.17492377  0.        ]
 [ 0.84482759  0.18738778 -0.61906222  0.20698637  0.        ]
 [ 0.86206897  0.18738778 -0.61906222  0.20698637  0.        ]
 [ 0.87931034  0.18738778 -0.61906222  0.20698637  0.        ]
 [ 0.89655172  0.18738778 -0.61906222  0.23057109  0.        ]
 [ 0.9137931   0.18738778 -0.61906222  0.23723288  0.        ]
 [ 0.93103448  0.18738778 -0.61906222  0.26468456  0.        ]
 [ 0.94827586  0.18738778 -0.61906222  0.27368137  0.        ]
 [ 0.96551724  0.18738778 -0.61906222  0.29954198  0.        ]
 [ 0.98275862  0.18738778 -0.61906222  0.3441682   0.        ]
 [ 0.98275862  0.18738778 -0.61906222  0.0030797   0.00227502]
 [ 0.98275862  0.18738778 -0.61

[0.00305438 0.00225749]
[[ 0.93103448  0.18738778 -0.61906222  0.26468456  0.        ]
 [ 0.94827586  0.18738778 -0.61906222  0.27368137  0.        ]
 [ 0.96551724  0.18738778 -0.61906222  0.29954198  0.        ]
 [ 0.98275862  0.18738778 -0.61906222  0.3441682   0.        ]
 [ 0.98275862  0.18738778 -0.61906222  0.0030797   0.00227502]
 [ 0.98275862  0.18738778 -0.61906222  0.00306912  0.00227213]
 [ 0.98275862  0.18738778 -0.61906222  0.00306667  0.00226991]
 [ 0.98275862  0.18738778 -0.61906222  0.00306514  0.00226823]
 [ 0.98275862  0.18738778 -0.61906222  0.00306383  0.00226681]
 [ 0.98275862  0.18738778 -0.61906222  0.00306261  0.00226559]
 [ 0.98275862  0.18738778 -0.61906222  0.00306149  0.00226455]
 [ 0.98275862  0.18738778 -0.61906222  0.0030605   0.00226367]
 [ 0.98275862  0.18738778 -0.61906222  0.00305962  0.00226294]
 [ 0.98275862  0.18738778 -0.61906222  0.00305885  0.00226234]
 [ 0.98275862  0.18738778 -0.61906222  0.00305819  0.00226186]
 [ 0.98275862  0.18738778 -0.61

[0.0030524  0.00225589]
[[ 0.98275862  0.18738778 -0.61906222  0.00306667  0.00226991]
 [ 0.98275862  0.18738778 -0.61906222  0.00306514  0.00226823]
 [ 0.98275862  0.18738778 -0.61906222  0.00306383  0.00226681]
 [ 0.98275862  0.18738778 -0.61906222  0.00306261  0.00226559]
 [ 0.98275862  0.18738778 -0.61906222  0.00306149  0.00226455]
 [ 0.98275862  0.18738778 -0.61906222  0.0030605   0.00226367]
 [ 0.98275862  0.18738778 -0.61906222  0.00305962  0.00226294]
 [ 0.98275862  0.18738778 -0.61906222  0.00305885  0.00226234]
 [ 0.98275862  0.18738778 -0.61906222  0.00305819  0.00226186]
 [ 0.98275862  0.18738778 -0.61906222  0.00305764  0.00226147]
 [ 0.98275862  0.18738778 -0.61906222  0.0030572   0.00226117]
 [ 0.98275862  0.18738778 -0.61906222  0.00305686  0.00226094]
 [ 0.98275862  0.18738778 -0.61906222  0.00305661  0.00226075]
 [ 0.98275862  0.18738778 -0.61906222  0.00305646  0.0022606 ]
 [ 0.98275862  0.18738778 -0.61906222  0.00305638  0.00226045]
 [ 0.98275862  0.18738778 -0.61

KeyboardInterrupt: 

In [27]:
train_df_on_idx = train_df[(all_df["Country/Region"] == idx[1]) &
                           (all_df["Province/State"] == idx[0])]

In [69]:
train_df_on_idx = train_df[(all_df["Country/Region"] == idx[1]) &
                           (all_df["Province/State"] == idx[0])]
print(train_df_on_idx["Date"].values)
int(1000000000*(datetime.timedelta(days=day) + test_df_on_idx["Date"].min()).timestamp()) in train_df_on_idx["Date"].values

['2020-01-22T00:00:00.000000000' '2020-01-23T00:00:00.000000000'
 '2020-01-24T00:00:00.000000000' '2020-01-25T00:00:00.000000000'
 '2020-01-26T00:00:00.000000000' '2020-01-27T00:00:00.000000000'
 '2020-01-28T00:00:00.000000000' '2020-01-29T00:00:00.000000000'
 '2020-01-30T00:00:00.000000000' '2020-01-31T00:00:00.000000000'
 '2020-02-01T00:00:00.000000000' '2020-02-02T00:00:00.000000000'
 '2020-02-03T00:00:00.000000000' '2020-02-04T00:00:00.000000000'
 '2020-02-05T00:00:00.000000000' '2020-02-06T00:00:00.000000000'
 '2020-02-07T00:00:00.000000000' '2020-02-08T00:00:00.000000000'
 '2020-02-09T00:00:00.000000000' '2020-02-10T00:00:00.000000000'
 '2020-02-11T00:00:00.000000000' '2020-02-12T00:00:00.000000000'
 '2020-02-13T00:00:00.000000000' '2020-02-14T00:00:00.000000000'
 '2020-02-15T00:00:00.000000000' '2020-02-16T00:00:00.000000000'
 '2020-02-17T00:00:00.000000000' '2020-02-18T00:00:00.000000000'
 '2020-02-19T00:00:00.000000000' '2020-02-20T00:00:00.000000000'
 '2020-02-21T00:00:00.000

/Users/yamaguchir/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


False

In [72]:
test_df_on_idx.Date.min()

Timestamp('2020-03-12 00:00:00')

In [28]:
train_df_on_idx

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,Date_num
2006,3163,Alberta,Canada,0.299629,-0.647647,2020-01-22,0.000000,0.0,0.000000
2007,3164,Alberta,Canada,0.299629,-0.647647,2020-01-23,0.000000,0.0,0.017241
2008,3165,Alberta,Canada,0.299629,-0.647647,2020-01-24,0.000000,0.0,0.034483
2009,3166,Alberta,Canada,0.299629,-0.647647,2020-01-25,0.000000,0.0,0.051724
2010,3167,Alberta,Canada,0.299629,-0.647647,2020-01-26,0.000000,0.0,0.068966
2011,3168,Alberta,Canada,0.299629,-0.647647,2020-01-27,0.000000,0.0,0.086207
2012,3169,Alberta,Canada,0.299629,-0.647647,2020-01-28,0.000000,0.0,0.103448
2013,3170,Alberta,Canada,0.299629,-0.647647,2020-01-29,0.000000,0.0,0.120690
2014,3171,Alberta,Canada,0.299629,-0.647647,2020-01-30,0.000000,0.0,0.137931
2015,3172,Alberta,Canada,0.299629,-0.647647,2020-01-31,0.000000,0.0,0.155172


In [24]:

all_df = train_df.fillna({"Province/State": "NAN"})

In [45]:
int((datetime.timedelta(days=1) + test_df_on_idx["Date"].min()).timestamp()) in train_df_on_idx.Date.values.tolist()

False

In [64]:
int(1000000000*(datetime.timedelta(days=1) + test_df_on_idx["Date"].min()).timestamp()) in train_df_on_idx.Date.values.tolist()

True

In [48]:
train_df_on_idx.Date.values.tolist()[0]

1579651200000000000

In [38]:
(datetime.timedelta(days=1) + test_df_on_idx["Date"].min()).values

AttributeError: 'Timestamp' object has no attribute 'values'

In [41]:
test_df_on_idx["Date"].values

array(['2020-03-12T00:00:00.000000000', '2020-03-13T00:00:00.000000000',
       '2020-03-14T00:00:00.000000000', '2020-03-15T00:00:00.000000000',
       '2020-03-16T00:00:00.000000000', '2020-03-17T00:00:00.000000000',
       '2020-03-18T00:00:00.000000000', '2020-03-19T00:00:00.000000000',
       '2020-03-20T00:00:00.000000000', '2020-03-21T00:00:00.000000000',
       '2020-03-22T00:00:00.000000000', '2020-03-23T00:00:00.000000000',
       '2020-03-24T00:00:00.000000000', '2020-03-25T00:00:00.000000000',
       '2020-03-26T00:00:00.000000000', '2020-03-27T00:00:00.000000000',
       '2020-03-28T00:00:00.000000000', '2020-03-29T00:00:00.000000000',
       '2020-03-30T00:00:00.000000000', '2020-03-31T00:00:00.000000000',
       '2020-04-01T00:00:00.000000000', '2020-04-02T00:00:00.000000000',
       '2020-04-03T00:00:00.000000000', '2020-04-04T00:00:00.000000000',
       '2020-04-05T00:00:00.000000000', '2020-04-06T00:00:00.000000000',
       '2020-04-07T00:00:00.000000000', '2020-04-08

In [35]:
train_df_on_idx.Date.values

array(['2020-01-22T00:00:00.000000000', '2020-01-23T00:00:00.000000000',
       '2020-01-24T00:00:00.000000000', '2020-01-25T00:00:00.000000000',
       '2020-01-26T00:00:00.000000000', '2020-01-27T00:00:00.000000000',
       '2020-01-28T00:00:00.000000000', '2020-01-29T00:00:00.000000000',
       '2020-01-30T00:00:00.000000000', '2020-01-31T00:00:00.000000000',
       '2020-02-01T00:00:00.000000000', '2020-02-02T00:00:00.000000000',
       '2020-02-03T00:00:00.000000000', '2020-02-04T00:00:00.000000000',
       '2020-02-05T00:00:00.000000000', '2020-02-06T00:00:00.000000000',
       '2020-02-07T00:00:00.000000000', '2020-02-08T00:00:00.000000000',
       '2020-02-09T00:00:00.000000000', '2020-02-10T00:00:00.000000000',
       '2020-02-11T00:00:00.000000000', '2020-02-12T00:00:00.000000000',
       '2020-02-13T00:00:00.000000000', '2020-02-14T00:00:00.000000000',
       '2020-02-15T00:00:00.000000000', '2020-02-16T00:00:00.000000000',
       '2020-02-17T00:00:00.000000000', '2020-02-18

In [32]:
test_df_on_idx["Date"].min()

Timestamp('2020-03-12 00:00:00')

In [ ]:
submit_df = pd.read_csv(data_dir+"submission.csv", index_col=0)

In [ ]:
submit_df

In [ ]:
cases = []
fatals = []
for i in range(len(results)):
    n = results[i][0] 
    f = results[i][1]
    try:
        cases.append(int(n))
    except:
        cases.append(0)
    
    try:
        fatals.append(int(f))
    except:
        fatals.append(0)

In [ ]:
submit_df["ConfirmedCases"] = cases
submit_df["Fatalities"] = fatals

In [ ]:
submit_df.to_csv("submission.csv")